<a href="https://colab.research.google.com/github/eletorres/MachineLearning/blob/main/ProyectoML_LTorres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import IPython.core.display
import matplotlib ## paquete necesario para poder hacer gráficas

def apply_styles(): ### especificamos los parámetros para los gráficos
    matplotlib.rcParams['font.size'] = 12
    matplotlib.rcParams['figure.figsize'] = (18, 6)
    matplotlib.rcParams['lines.linewidth'] = 1

apply_styles() ### finalmente aplicamos el estilo

In [ ]:
### éstos elementos configuran fuente tamaño color de la letra porcentaje etc.
%%html
<style type="text/css">
@import url('https://fonts.googleapis.com/css?family=Playfair+Display');


div.text_cell_render {font-family: 'Playfair Display', serif; color: #13213b; line-height: 145%; font-size:16px;}
</style>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [ ]:
import warnings ## desactivan los mensajes para ciertos advertencias en las funciones
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
## la siguiente línea indica la dirección donde se encuentran los datos que queremos cargarla siguiente línea indica la dirección donde se encuentran los datos que queremos cargar
### si te fijas bien la dirección corresponde precisamente al proyecto github que acabamos de cargar
## solamente que apunta a un archivo CSV

url="https://raw.githubusercontent.com/castudil/Machine-Learning/main/data/GMAA-040218.csv"
## Básicamente tomamos ese archivo y lo cargamos usando un método que lo transfiere hacia un data frame
df = pd.read_csv(url, header=None, skiprows=6, parse_dates=[0], names=['period', 'value'])
df.value.astype(int, copy=False);